In [148]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import metrics

In [86]:
data_crops = pd.read_csv("apy.csv")
data_crops

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0
...,...,...,...,...,...,...,...
246086,West Bengal,PURULIA,2014,Summer,Rice,306.0,801.0
246087,West Bengal,PURULIA,2014,Summer,Sesamum,627.0,463.0
246088,West Bengal,PURULIA,2014,Whole Year,Sugarcane,324.0,16250.0
246089,West Bengal,PURULIA,2014,Winter,Rice,279151.0,597899.0


In [87]:
print(len(data_crops['State_Name'].unique()))  
print(len(data_crops['District_Name'].unique()))
print(len(data_crops['Season'].unique()))
print(len(data_crops['Crop'].unique()))

33
646
6
124


In [88]:
'''While converting the non-numericdata (categorical) to numeric data.....we ought to compare one-hot encoding 
and binary encoding
Fields considered are
1.states
2.districts
3.crop
4.seasons
'''

'While converting the non-numericdata (categorical) to numeric data.....we ought to compare one-hot encoding \nand binary encoding\nFields considered are\n1.states\n2.districts\n3.crop\n4.seasons\n'

In [135]:
#ONE HOT ENCODING

ohe = ce.OneHotEncoder()
d_1 = ohe.fit_transform(data_crops)
d_1.shape

(246091, 812)

In [136]:
#BINARY ENCODING

be = ce.BinaryEncoder()
data = be.fit_transform(data_crops)   #if none columns are specified the method considers all the string attributes by default
data.shape

(246091, 33)

In [137]:
'''reducing the column count from 812 to 33 by using the binary encoding'''


#converting the dataframe to arrays
Y = data.iloc[:,32]
X = data.drop('Production',axis = 'columns')
Y = Y.to_frame()
print(type(X))
print(type(Y))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [138]:
#finding and replacing the missing values

print(Y.isnull().sum())
print(X.isnull().sum())

# Having missing values in production 
Y.Production.fillna(Y.Production.mean(),inplace=True)

Production    3730
dtype: int64
State_Name_0        0
State_Name_1        0
State_Name_2        0
State_Name_3        0
State_Name_4        0
State_Name_5        0
State_Name_6        0
District_Name_0     0
District_Name_1     0
District_Name_2     0
District_Name_3     0
District_Name_4     0
District_Name_5     0
District_Name_6     0
District_Name_7     0
District_Name_8     0
District_Name_9     0
District_Name_10    0
Crop_Year           0
Season_0            0
Season_1            0
Season_2            0
Season_3            0
Crop_0              0
Crop_1              0
Crop_2              0
Crop_3              0
Crop_4              0
Crop_5              0
Crop_6              0
Crop_7              0
Area                0
dtype: int64


In [142]:
#spliting the data into X and Y

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, train_size = 0.8, test_size = 0.2)


In [151]:
# Defining the Random Regressor

regressor = RandomForestRegressor(n_estimators = 10,random_state = 0)
regressor.fit(X_train,Y_train)
print("trained")
print(X_train)
print(Y_train)

/Users/mohankalyanveeraghanta/Software-Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


trained
        State_Name_0  State_Name_1  State_Name_2  State_Name_3  State_Name_4  \
146467             0             0             1             0             1   
9422               0             0             0             0             0   
99523              0             0             1             0             0   
42893              0             0             0             0             1   
114761             0             0             1             0             0   
...              ...           ...           ...           ...           ...   
55556              0             0             0             0             1   
17833              0             0             0             0             1   
193885             0             0             1             1             1   
230007             0             0             1             1             1   
162890             0             0             1             1             0   

        State_Name_5  State_Nam

In [152]:
#testing

actual_prediction = regressor.predict(X_test)

actual_prediction
Y_test

In [153]:
calculating the error rate

print('Mean Squared Error:', metrics.mean_squared_error(Y_test, actual_prediction))